## Imports

In [1]:
import pandas as pd
import numpy as np

## Files path

In [3]:
FILE_PATH = "C:/Users/Stephen/Desktop/EURUSD_M5.csv"

## Read dataset

In [13]:
df = pd.read_csv(FILE_PATH)

In [14]:
df.head()

,Open,High,Low,Close,Volume,Date,Time
0,1.27380,1.28220,1.27180,1.2814,38504,2008-12-04,16:00:00
1,1.28155,1.28475,1.27995,1.2838,28903,2008-12-04,16:30:00
2,1.28380,1.28400,1.28115,1.2829,18663,2008-12-04,17:00:00
3,1.28295,1.28305,1.27860,1.2792,12610,2008-12-04,17:30:00
4,1.27915,1.28025,1.27860,1.2793,11970,2008-12-04,18:00:00


## Clean dataset

In [4]:
data = pd.read_csv("Dataset\EURUSD\EURUSD_M30.csv")

# Split the single column into multiple columns using tab as the delimiter
data_split = data['Time\tOpen\tHigh\tLow\tClose\tVolume'].str.split('\t', expand=True)

# Retain only the first 6 columns and assign meaningful column names
data_split = data_split.iloc[:, :6]
data_split.columns = ['DateTime', 'Open', 'High', 'Low', 'Close', 'Volume']

# Split DateTime into separate Date and Time columns
data_split[['Date', 'Time']] = data_split['DateTime'].str.split(' ', expand=True)
data_split.drop(columns=['DateTime'], inplace=True)

# Convert data types
# Parse datetime for the Date column
data_split['Date'] = pd.to_datetime(data_split['Date'], errors='coerce')

# Convert price and volume columns to numeric types
numeric_columns = ['Open', 'High', 'Low', 'Close', 'Volume']
for col in numeric_columns:
    data_split[col] = pd.to_numeric(data_split[col], errors='coerce')

# Drop rows with missing or invalid values
data_cleaned = data_split.dropna()

# Save the cleaned dataset to a new CSV file
output_file = "EURUSD_M30_cleaned.csv"  # Replace with the desired output file name
data_cleaned.to_csv(output_file, index=False)

print(f"Dataset cleaned and saved to {output_file}.")

<>:1: SyntaxWarning: invalid escape sequence '\E'
<>:1: SyntaxWarning: invalid escape sequence '\E'
C:\Users\Stephen\AppData\Local\Temp\ipykernel_27652\198518384.py:1: SyntaxWarning: invalid escape sequence '\E'
  data = pd.read_csv("Dataset\EURUSD\EURUSD_M30.csv")


Dataset cleaned and saved to EURUSD_M30_cleaned.csv.


## Add RSI14 and SMAs to dataset

In [3]:
# Load the cleaned dataset
FILE_PATH = "./Dataset/EURUSD/EURUSD_M30_cleaned.csv"  # Replace with the actual file path
data = pd.read_csv(FILE_PATH)

# Ensure the dataset is sorted by Date and Time
data['Date'] = pd.to_datetime(data['Date'])
data['DateTime'] = pd.to_datetime(data['Date'].astype(str) + ' ' + data['Time'])
data = data.sort_values('DateTime').reset_index(drop=True)

# Calculate SMAs
data['SMA200'] = data['Close'].rolling(window=200).mean()
data['SMA100'] = data['Close'].rolling(window=100).mean()
data['SMA50'] = data['Close'].rolling(window=50).mean()
data['SMA13'] = data['Close'].rolling(window=13).mean()

# Calculate RSI14
# Define a function to compute RSI
def calculate_rsi(series, period=14):
    delta = series.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

data['RSI14'] = calculate_rsi(data['Close'], period=14)

# Drop rows with missing values (e.g., the first 100 rows that lack SMA100)
data = data.dropna().reset_index(drop=True)

# Save the dataset with the new features
output_file = "EURUSD_M30_feature_v.2.csv"  # Replace with the desired output file name
data.to_csv(output_file, index=False)

print(f"SMAs and RSI14 added, cleaned, and saved to {output_file}.")


SMAs and RSI14 added, cleaned, and saved to EURUSD_M30_feature_v.2.csv.
